## <center> Аттестация. Практическое задание в Jupiter

В данном блоке вам предстоит решить задачу, которую автор когда-то решал на одном из своих собеседований.
Вам предоставлена таблица с данными (скачать файл можно на платформе) о ежедневной выработке энергии солнечными батареями. Информация в ней содержится в двух столбцах: 
* date - дата наблюдения
* cum_power - накопленная (кумулятивная) выработка энергии на указанный день

То есть перед вами классический пример временного ряда: числовой показатель, зависящий от времени. Давайте с ним поработаем.


In [118]:
# 0
import pandas as pd
import numpy as np

# Чтение файла, должен находиться в том же каталоге, что и файл c кодом
data = pd.read_csv('solarpower.csv')

# Проверка структуры, отмечаем что пропусков нет
display(data.info())
display(data.head(5))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 664 entries, 0 to 663
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   date       664 non-null    object 
 1   cum_power  664 non-null    float64
dtypes: float64(1), object(1)
memory usage: 10.5+ KB


None

,date,cum_power
0,2017-01-01,20111.0
1,2017-01-02,20112.0
2,2017-01-03,20115.0
3,2017-01-04,20117.0
4,2017-01-05,20119.0


1. В нашем временном ряде есть несколько пропусков, они обозначены числом -1. Заполните пропуски средним арифметическим между двумя соседними наблюдениями: </p>
    $$x^*_i  = \frac{x_{i-1} - x_{i+1}}{2}$$ 
    
Совет: проще всего будет сделать это, используя цикл по индексам элементов столбца cum_power, но вы можете предложить более изящное решение.


In [119]:
# 1
# Проверка наличия таких значений
filtered = data[data['cum_power'] == -1]
display(filtered)

# Получили всего две строки, вычислим необходимые значения конкретно для них
indexes = [45, 647]
for i in indexes:
    data['cum_power'].iloc[i] = (data['cum_power'].iloc[i-1] + data['cum_power'].iloc[i+1]) / 2


,date,cum_power
45,2017-02-15,-1.0
647,2018-10-10,-1.0


C:\Users\Dell\anaconda3\envs\Nadin_tiger\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



2. Теперь, когда таблица заполнена, создайте столбец day_power - ежедневный прирост выработки солнечной энергии.

    После чего создайте данные о ежемесячной выработке энергии на каждый год. 
    
Совет: для того, чтобы выделить из даты год и месяц проще всего воспользоваться методом аксессора dt.to_period('M'), который позволяет выделять периоды, в данном случае 'М' - месяц.

In [120]:
# 2 
# Day power - прирост выработки энергии за день
data['day_power'] = data['cum_power'] - data['cum_power'].shift()

# Первое значение в новом столбце пропущено в случае использования shift(), заменим среднеарифметическим
# подставлять значение из cum_power той же строки некорректно, значение будет слишком сильно отличаться от остальных
mean_power = data['day_power'].mean()
display('Среднеарифметическое значение по столбцу day_power:', round(mean_power, 2))

# Тоже не очень нравится, возможно мощности расли со временем, соседние значения отличаются в несколько раз
# Заполним пропуск значением из следующей строки
data['day_power'].iloc[0] = data['day_power'].iloc[1]

# Столбец date имеет тип object, сначала преобразуем в формат даты
# И тогда мы сможем выделить из даты Month - месяц
data['date'] = pd.to_datetime(data['date'])
data['month'] = data['date'].dt.to_period('M')
display(data.head(5))

'Среднеарифметическое значение по столбцу day_power:'

12.08

,date,cum_power,day_power,month
0,2017-01-01,20111.0,1.0,2017-01
1,2017-01-02,20112.0,1.0,2017-01
2,2017-01-03,20115.0,3.0,2017-01
3,2017-01-04,20117.0,2.0,2017-01
4,2017-01-05,20119.0,2.0,2017-01


3. Постройте график временного ряда - ежемесячной выработки солнечной энергии. Не забудьте подписать оси и сам график.
Укажите в какие сезоны наблюдается подъем, а в какие спад?

In [121]:
# 3
# Будем использовать столбчатую диаграмму plotly express, так как периоды времени больше похожи на категории
import plotly.express as px

# Сгруппируем данные ежедневной выработки энергии по месяцам
grouped = data.groupby('month', as_index=False)['day_power'].sum()

# Для корректного отображения данных на графике изменим тип данных Period на строковый и изменим название столбца
grouped['month'] = grouped['month'].apply(lambda x: str(x))
grouped['monthly_power'] = grouped['day_power']
grouped = grouped.drop('day_power', axis=1)

fig = px.bar(
    data_frame=grouped,
    x=grouped['month'],
    y=grouped['monthly_power'],
    height=500, #высота
    width=700, #ширина
    title='Зависимость ежемесячной выработки солнечной энергии от времени'
)

# Для загрузки на Github использую выведенный ранее график, так как вывод графиков plotly express при этом не работает
fig.write_image('solar.png')
fig.show()


![](solar.png)

График показывает зависимость уровня выработки солнечной энергии от времени года - пиковые месяцы с мая по июль, минимальный уровень достигается в период с ноября по январь. При этом в 2017 году самый пик был в июне, а в 2018 эта закономерность нарушилась - в июне было выработано меньше энергии, чем в мае и июле. Этот момент требует дополнительного внимания. Возможно, действительно что-то произошло на объекте, либо есть неявные ошибки в данных.